####  Two Different Kinds of Coin Flip

The first scenario was covered in class:  we have two coins, **C1** and **C2** that can flip either *heads* or *tails*.
We know that either both coins are fair, or they both are not fair.  If a coin is fair, it flips heads 50\% of the time and tails 50\% of the time.  If a coin is not fair, it flips heads 90\% of the time and tails 10\% of the time.  Initially you have no information about whether or not the coins are fair.

Construct the network modeling this scenario, and use it to determine the probability that both coins will flip *heads* if both are flipped exactly once.


In [7]:
from pomegranate import *

# Define the probability distributions
fairness_distribution = DiscreteDistribution({'Fair': 0.5, 'Unfair': 0.5})
coin_flip_distribution_fair = ConditionalProbabilityTable(
    [['Fair', 'Heads', 0.5],
     ['Fair', 'Tails', 0.5],
     ['Unfair', 'Heads', 0.9],
     ['Unfair', 'Tails', 0.1]],
    [fairness_distribution])

# Define the nodes
fairness_node = State(fairness_distribution, name="Fairness")
c1_node = State(coin_flip_distribution_fair, name="C1")
c2_node = State(coin_flip_distribution_fair, name="C2")

# Build the model
model = BayesianNetwork("Two Coin Flips")
model.add_states(fairness_node, c1_node, c2_node)
model.add_edge(fairness_node, c1_node)
model.add_edge(fairness_node, c2_node)
model.bake()

In [8]:
# Compute and print the probability that both coins will flip heads
prob_both_heads = model.probability([['Fair', 'Heads', 'Heads'],
                                     ['Unfair', 'Heads', 'Heads']])
print(f"Probability of both coins flipping heads: {prob_both_heads}")

Probability of both coins flipping heads: [0.125 0.405]


The definition of unconditional indepencence says that C3 is independent of C4 if $P(C3 | C4) = P(C3)$.
Is it the case that in scenario, C3 and C4 are unconditionally independent?  Why or why not?


In the given scenario, C3 and C4 are described as being unconditionally independent. This means that the probability of C3 flipping heads or tails is not influenced by the outcome of C4, and vice versa. According to the definition of unconditional independence,
P(C3∣C4)=P(C3), and similarly,
P(C4∣C3)=P(C4). So, yes, C3 and C4 are unconditionally independent because their outcomes do not influence each other, adhering to the definition of unconditional independence.

One of the properties of conditional independence is that if C1 and C2 are independent conditioned on F, then if you know the state of F, knowledge about the state of C1 does not change the probability of C2.  That is,

$P(C1 | F, C2) = P(C1 | F)$

and

$P(C1 | \neg F, C2) = P(C1 | \neg F)$

Recall that by definition, in a Bayesian network, children of a common parent are independent conditioned on their parent.  Demonstrate that his property holds in the Coins network you just built.

In this scenario, F has two states: Fair (F) and Unfair (¬F). The probabilities of C1 and C2 flipping heads or tails are conditioned on F. If F is known, the flips of C1 and C2 are independent of each other.

  - Knowing that C1 flipped heads (or tails) does not change the probability of C2 flipping heads or tails, which remains 0.5 for heads and 0.5 for tails because both coins adhere to F's fairness.

  - Similarly, if we know F is Unfair, the outcome of C1 does not affect the probability of C2's flip. Both are influenced solely by ¬F, with a 0.9 probability for heads and 0.1 for tails.

Observing the outcome of C1 does not provide additional information about C2's flip once we know the state of F. This is because the conditional probabilities of C1 and C2's outcomes are determined entirely by F, making C1 and C2 conditionally independent of each other given F.

#### Second scenario

Now consider a different scenario:  a child node with two parents.

* C3 is a coin that flips *heads* 75\% of the time and *tails* 25\% of the time
* C4 is a coin that flips *heads* 35\% of the time and *tails* 65\% of the time
* C3 and C4 are independent in the sense that no "bias" machine that produced them both
* C3 and C4 are both flipped a single time
* L is a light that is supposed go *on* if and only if both C3 and C4 flip heads
* But L sometimes malfunctions
   * .1\% of the time L does not go *on* even though both C3 and C4 flipped heads
   * .01\% of the time L goes on even though C3 and C4 did not both flip heads

In [9]:
# Coin C3 and C4 distributions
c3_distribution = DiscreteDistribution({'Heads': 0.75, 'Tails': 0.25})
c4_distribution = DiscreteDistribution({'Heads': 0.35, 'Tails': 0.65})

# Light (L) behavior based on C3 and C4 flips
l_distribution = ConditionalProbabilityTable(
    [['Heads', 'Heads', 'On', 0.999],
     ['Heads', 'Heads', 'Off', 0.001],
     ['Heads', 'Tails', 'On', 0.0001],
     ['Heads', 'Tails', 'Off', 0.9999],
     ['Tails', 'Heads', 'On', 0.0001],
     ['Tails', 'Heads', 'Off', 0.9999],
     ['Tails', 'Tails', 'On', 0.0001],
     ['Tails', 'Tails', 'Off', 0.9999]],
    [c3_distribution, c4_distribution])

# Define the nodes
c3_node = State(c3_distribution, name="C3")
c4_node = State(c4_distribution, name="C4")
l_node = State(l_distribution, name="Light")

# Build the Bayesian Network
model = BayesianNetwork("Coin and Light Scenario")
model.add_states(c3_node, c4_node, l_node)
model.add_edge(c3_node, l_node)
model.add_edge(c4_node, l_node)
model.bake()

In [21]:
# Given no other information, what's the probability that the light will go on?

probability_light_on = model.probability([['Heads', 'Heads', 'On'],
                                            ['Heads', 'Tails', 'On'],
                                            ['Tails', 'Heads', 'On'],
                                            ['Tails', 'Tails', 'On']])

print(probability_light_on)

[2.622375e-01 4.875000e-05 8.750000e-06 1.625000e-05]


In [18]:
probability_light_on = sum([2.622375e-01, 4.875000e-05, 8.750000e-06, 1.625000e-05])
print("Probability the light will go on:", probability_light_on)

Probability the light will go on: 0.26231125


In [22]:
# Suppose you observe the light on, what is the probability that coin4 flipped heads?

observations = {'Light': 'On'}
probabilities = model.predict_proba(observations)

print("Probability C4 flipped heads given the light is on:", probabilities[1].parameters[0]['Heads'])

Probability C4 flipped heads given the light is on: 0.9997522027743754


In [23]:
# What is the prior probability that coin4 flips heads? Compute and print the probability.

print("Prior probability that C4 flips heads:", c4_distribution.parameters[0]['Heads'])

Prior probability that C4 flips heads: 0.35


In [24]:
# Now you observe that coin3 flipped heads;  does that change your believe that coin4 flipped heads?

# Assuming implementation of predict_proba or equivalent functionality
observations = {'C3': 'Heads'}
probabilities = model.predict_proba(observations)

print("Belief about C4 flipping heads given C3 flipped heads:", probabilities[1].parameters[0]['Heads'])

Belief about C4 flipping heads given C3 flipped heads: 0.3500000000000001


Based on the output above, this indicates that observing Coin 3 (C3) flipping heads does not change the belief about Coin 4 (C4) flipping heads; the probability remains at 35.00000000000001%, essentially 35%.

  - C3 and C4 are independent of each other.
  - The prior probability of C4 flipping heads was given at 35%. After observing C3 flip heads, the calculated probability of C4 flipping heads remains at 35%.

By the definition of independence, random variable C3 is (unconditionally) independent of C4 if
$P(C3 | C4) = P(C3)$

Is it true then that C3 and C4 are unconditionally independent even though they have a common child?
How does this contrast with the situation in the first model where C1 and C2 were not

C3 and C4 are described as independent coins, meaning the outcome of one does not influence the outcome of the other. This independence is reflected in the statement
P(C3∣C4)=P(C3), which holds true by the scenario's design. The flip of C4 does not provide any information about the flip of C3 and vice versa, so their probabilities remain unchanged regardless of knowledge about the other's outcome.

In addition, this independence holds true despite them having a common child (L) in the Bayesian network, which introduces conditional dependence when the state of L is known but does not affect their unconditional independence.

As for C1 and C2, the coins were either both fair or both unfair, implying a dependency between their states. Their fairness was linked by a common factor (their fairness/unfairness), which directly influenced their probabilities of flipping heads or tails. Knowledge about the state of one coin (fair or unfair) would directly inform the state of the other, making them conditionally dependent based on their fairness.



Return to the definition of conditional independence above, where C1 and C2 were independent conditioned on their common parent.  Are C3 and C4 independent conditioned on their common child?

In the context of Bayesian networks, C3 and C4 are initially independent, meaning the outcome of one does not affect the outcome of the other. However, when considering their common child, L (the light), this independence is conditional. Before knowing L's state, C3 and C4's outcomes are independent. Yet, once L's state is known, C3 and C4 become conditionally dependent on each other due to the information L's state provides about their joint outcomes.


Summarize the different in independence assumptions between the two scenarios
* In the first scenario there were two children with a common parent
* In the second scenario there were two parents with a common child

First Scenario (Two Children with a Common Parent): This setup involves two nodes (C1 and C2) linked by a common parent representing an underlying condition affecting both (e.g., the fairness of coins). The key feature is conditional independence: once the condition of the parent (fairness) is known, the outcomes of C1 and C2 are independent of each other. Their behavior is dictated by the parent's state, but not by each other's outcomes directly.

Second Scenario (Two Parents with a Common Child): Here, two independent nodes (C3 and C4) act as parents to a common child node (L), whose state depends on the outcomes of C3 and C4. Initially, C3 and C4 are unconditionally independent; their actions don't influence each other. However, conditional dependence emerges when considering the state of L: the state of the child introduces a link between C3 and C4, making their outcomes dependent when analyzed through the lens of L's state.

Direction of Influence: In the first scenario, influence flows from the parent to the children, and knowledge of the parent's state reveals the children's independence. The second observes the influence of the child's status on its parents, indicating their conditional dependence.

Overall, understanding conditional independence versus conditional dependency is critical. The first scenario's conditional independence is revealed by knowledge of the parent's state, whereas the second scenario's conditional dependency arises from observations of the child's state.

